In [ ]:
from nerf import utils
from jax import device_put
import jax.numpy as jnp
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from visualhull_blender import *


FLAGS.config = "configs/blender"
FLAGS.data_dir = "data/nerf_synthetic/hotdog"
FLAGS.vh_test = True

FLAGS.alpha_bkgd = True
FLAGS.num_rgb_channels = 4
if FLAGS.config is not None:
    utils.update_flags(FLAGS)

vsize = FLAGS.vsize

In [ ]:
target = FLAGS.data_dir.split("/")[-1]

# dataset = PureDataset("train", FLAGS)
# if FLAGS.dataset == "blender":
#     dataset.images = dataset.images.reshape(-1,800,800,4)
#     dataset.rays = dataset.rays._replace(origins = dataset.rays.origins.reshape(-1,800,800,3))
#     dataset.rays = dataset.rays._replace(directions = dataset.rays.directions.reshape(-1,800,800,3))
#     dataset.rays = dataset.rays._replace(viewdirs = dataset.rays.viewdirs.reshape(-1,800,800,3))
# elif FLAGS.dataset == "llff":
#     dataset.images = dataset.images.reshape(-1,756,1008,3)
#     dataset.rays = dataset.rays._replace(origins = dataset.rays.origins.reshape(-1,756,1008,3))
#     dataset.rays = dataset.rays._replace(directions = dataset.rays.directions.reshape(-1,756,1008,3))
#     dataset.rays = dataset.rays._replace(viewdirs = dataset.rays.viewdirs.reshape(-1,756,1008,3))

dataset = PureDataset("train", FLAGS)
dataset.images = dataset.images.reshape(-1,800,800,FLAGS.num_rgb_channels)
dataset.rays = dataset.rays._replace(
  origins=dataset.rays.origins.reshape(-1,800,800,3))
dataset.rays = dataset.rays._replace(
  directions=dataset.rays.directions.reshape(-1,800,800,3))
dataset.rays = dataset.rays._replace(
  viewdirs=dataset.rays.viewdirs.reshape(-1,800,800,3))

if FLAGS.vh_test:
    test_dataset = PureDataset("test", FLAGS)
    test_dataset.images = test_dataset.images.reshape(-1,800,800,FLAGS.num_rgb_channels)
    test_dataset.rays = test_dataset.rays._replace(
      origins=test_dataset.rays.origins.reshape(-1,800,800,3))
    test_dataset.rays = test_dataset.rays._replace(
      directions=test_dataset.rays.directions.reshape(-1,800,800,3))
    test_dataset.rays = test_dataset.rays._replace(
      viewdirs=test_dataset.rays.viewdirs.reshape(-1,800,800,3))

In [ ]:
# fig = plt.figure(figsize=(8, 8))
# ax = fig.add_subplot(111, projection='3d')
# ax.set_xlim(-5, 5)
# ax.set_ylim(-5, 5)
# ax.set_zlim(-5, 5)
# ax.view_init(elev=90, azim=90)
# # ax.view_init(elev=0, azim=0)
# ax.set_xlabel('x')
# ax.set_ylabel('y')
# ax.set_zlabel('z')

# for idx in range(0, len(dataset.rays.origins), 2):
#     o = dataset.rays.origins[idx]
#     d = dataset.rays.directions[idx]

#     # render lower limit（blue）
#     x_n, y_n, z_n = (o + d * t_n)[::16, ::16].reshape(-1, 3).T
#     ax.scatter(x_n, y_n, z_n, c='blue', s=0.1)

#     # render upper limit (green)
#     x_f, y_f, z_f = (o + d * t_f)[::16, ::16].reshape(-1, 3).T
#     ax.scatter(x_f, y_f, z_f, c='green', s=0.1)

In [ ]:
visualhull(dataset, test_dataset, target, dilation=47, thresh=100)